# Neural Network

Import required libraries

In [1]:
import numpy as np
import chess
import torch
import torch.nn as nn

Use neural network to evaluate positions


Notes:
- input; how do we encode the board
    - 12 by 64 input shape of bitboards


Todo:
- figure out number of layers
- figure out which activation function to use

Layers:
- input layer
    - represents state of the board
    - maybe 12 bitboards?
    - s = SAME_TEAM
    - o = OTHER_TEAM
    - input
        - S PAWN
        - S KNIGHT
        - S BISHOP
        - S ROOK
        - S QUEEN
        - S KING
        - O PAWN
        - O KNIGHT
        - O BISHOP
        - O ROOK
        - O QUEEN
        - O KING
- 
- output layer
    - one number that is the evalution


In [2]:
def bitboard(bb: int):
    s = 8 * np.arange(7, -1, -1, dtype=np.uint64)
    b = (bb >> s).astype(np.uint8)
    b = np.unpackbits(b, bitorder="little")
    return b

def bitboard_pretty(bb: np.array):
    for i in range(0, 64):
        if ((i) % 8 == 0):
            print()
        print(bb[i], end=" ")

In [3]:
board = chess.Board()
bitboard_pretty(bitboard(int(board.pieces(chess.PAWN, chess.WHITE))))


0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
1 1 1 1 1 1 1 1 
0 0 0 0 0 0 0 0 

In [4]:
"""
keras sequntial model 
optim: adam
loss: mse
Model architecture:
    linear of input 768 (12*64) bitboard

    i just made each layer 2/3 of the last layer plus the output
    https://www.heatonresearch.com/2017/06/01/hidden-layers.html 

    stockfish uses clamped relu(0, 1) for speed, but for now we can just use relu

    linear layer: input_shape: 768; output_shape: 513; activation function: relu
    linear layer: input_shape: 513; output_shape: 343; activation function: relu
    linear layer: input_shape: 343; output_shape: 229; activation function: relu
    linear layer: input_shape: 229; output_shape: 153; activation function: relu
    linear layer: input_shape: 153; output_shape: 1
    
    In c++, we will not have to do multiplications in the nn because it is just 0s and 1s
"""

'\nkeras sequntial model \noptim: adam\nloss: mse\nModel architecture:\n    later we could try a different type of input, but for now we will stick with this\n    the problem with 12*64 inputs is that if we change one thing in the position, the input is completely different, and in a search we usually look at similar moves, so if we do not do this\n    it is very bad for cache and it will run a lot slower.\n\n    linear of input 768 (12*64) bitboard\n    we could also try a 64 input layer with:\n    0.1 as pawn\n    0.2 as knight\n    0.3 as bishop\n    0.4 as rook\n    0.5 as queen\n    1 as king\n    this way if the one piece moves, only two inputs are changed\n\ni just made each layer 2/3 of the last layer plus the output\nhttps://www.heatonresearch.com/2017/06/01/hidden-layers.html \n\nlinear layer: input_shape: 768; output_shape: 513; activation function: relu, tanh, or sigmoid\nlinear layer: input_shape: 513; output_shape: 343; activation function: relu, tanh, or sigmoid\nlinear 

In [5]:
class Chess_Model(nn.Module):
    def __init__(self, activation_func: function) -> None:
        super().__init_()
        self.acivation_function = activation_func
        self.linear1 = nn.Linear(in_features=768, out_features=513)
        self.linear2 = nn.Linear(in_features=513, out_features=343)
        self.linear3 = nn.Linear(in_features=343, out_features=229)
        self.linear4 = nn.Linear(in_features=229, out_features=153)
        self.linear5 = nn.Linear(in_features=153, out_features=1)

    def forward(self, x):
        x = self.acivation_function(self.linear1(x))
        x = self.acivation_function(self.linear2(x))
        x = self.acivation_function(self.linear3(x))
        x = self.acivation_function(self.linear4(x))
        x = self.linear5(x)

        return x